In [ ]:
#libraries
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sklearn as sk
import urllib
import tarfile
import pandas as pd
import glob
import seaborn
import plotly.express as px
!pip install pytrends
from pytrends.request import TrendReq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16127 sha256=6caaff41c7e8406171f17ccd6638f31c0a22a45ebd263d3acfbb40dffa934596
  Stored in directory: /root/.cache/pip/wheels/e8/78/c8/18d4f4804753e14416809b365773220c48b41fe5387f2bb6b9
Successfully built pytrends



**Data Pre-processing**


In [ ]:
#Data pre-processing
#Load data

#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/MyDrive/Capstone_CMPS/PPP_Data
#!pwd
df = pd.read_csv('PPP_Fraud_Subsample.csv') #pick one dataset in order to prototype

In [ ]:
#Random test for google trends API

# Current issue is the trend data is not enough to get above 0
# even if the company exists.

pytrends = TrendReq(hl='en-US', tz=360) 

obs = df.sample()
keyword = [obs.iloc[0]['BorrowerName'],"Apple"]

pytrends.build_payload(keyword, timeframe="2019-12-14 2022-10-25", geo="US")

data = pytrends.interest_over_time()
data = data.reset_index()

fig = px.line(data, x="date", y=keyword, title='Keyword Web Search Interest Over Time')
fig.show()


In [ ]:
#%cd /content/drive/MyDrive/Capstone_CMPS
fraud_df = pd.read_csv("CustomData.csv")
#%cd /content/drive/MyDrive/Capstone_CMPS/PPP_Data
main_df = pd.read_csv("PPP_Fraud_Subsample.csv")

#read in two dataframes in order to prototype for main PPP dataset


In [ ]:
#supporting functions for text standardization
def removePunctuation(text):
  return str(text.translate(str.maketrans('', '', string.punctuation))) #removes all punctuation from string

def removeThreeChars(text):
  if (text[-3:] == ('LLC')) or (text[-3:] == ('INC')): #checks if last three characters have LLL or INC and retunrs string without those characters
    return text[:-3]
    #if no LLC or INC return text as is
  else:
    return text 

In [ ]:
import string
# standardize company name of dataframe in order to merge 
# this is just a prototype, the actual complete list of fraud claims compiled so far
# is in another file.

#steps:
# 1. Make everything Uppercase
# 2. Remove punctuation from string
# 3. remove whitespace from string -- string now has no spaces
# 4. Remove 'LLC' or 'INC' from string where applicable
# 5. merge data on newly created name

main_df['NewName'] = main_df['BorrowerName'].astype(str).str.upper().apply(removePunctuation).str.replace(' ', '').apply(removeThreeChars)
fraud_df['NewName'] = fraud_df['Company Name'].astype(str).str.upper().apply(removePunctuation).str.replace(' ', '').apply(removeThreeChars)
test_df = main_df.merge(fraud_df, how="inner",on="NewName")
test_df

,Unnamed: 0,Company Name_x,New Name,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NewName,Company Name_y,Fraud,Amount Taken,Case_ID,Date Charged
0,0,Hayventure LLC,HAYVENTURE,7045157304,04/30/2020,914.0,PPP,HAYVENTURE,506 S Spring St #1330,Los Angeles,...,Unanswered,NaN,239889.91,09/21/2021,HAYVENTURE,Hayventure LLC,1,"$5,151,497",1,2/25/2022
1,1,Pioneer LLC,PIONEER,4447147006,04/03/2020,875.0,PPP,PIONEER LLC,4041 36TH AVE S,FARGO,...,Non-Veteran,NaN,182143.66,01/20/2021,PIONEER,Pioneer LLC,1,"$5,151,497",1,2/25/2022
2,2,Pioneer LLC,PIONEER,3303968302,01/21/2021,709.0,PPS,PIONEER LLC,513 50th Steet,Hepler,...,Unanswered,NaN,104630.60,07/28/2021,PIONEER,Pioneer LLC,1,"$5,151,497",1,2/25/2022
3,3,Pioneer LLC,PIONEER,7274547002,04/07/2020,709.0,PPP,"PIONEER, LLC",513 50TH ST,HEPLER,...,Unanswered,NaN,104702.89,11/03/2020,PIONEER,Pioneer LLC,1,"$5,151,497",1,2/25/2022
4,4,XC International LLC,XCINTERNATIONAL,6288098600,03/23/2021,920.0,PPS,XC INTERNATIONAL LLC,PARK CENTER DR,COSTA MESA,...,Unanswered,NaN,NaN,NaN,XCINTERNATIONAL,XC International LLC,1,"$5,151,497",1,2/25/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,58,Sextant Marine Consulting LLC,SEXTANTMARINECONSULTING,7766297300,04/30/2020,455.0,PPP,SEXTANT MARINE CONSULTING LLC,1917 NW 82ND AVE,DORAL,...,Unanswered,NaN,151721.92,07/01/2021,SEXTANTMARINECONSULTING,Sextant Marine Consulting LLC,1,"$170,000",25,10/28/2021
61,59,Fabulous Appetizers LLC,FABULOUSAPPETIZERS,6702857810,06/02/2020,299.0,PPP,FABULOUS APPETIZERS,238 Lawrence Avenue,North Plainfield,...,Unanswered,NaN,NaN,NaN,FABULOUSAPPETIZERS,Fabulous Appetizers LLC,1,"$860,000",29,1/25/2022
62,60,Kremkov Industries LLC,KREMKOVINDUSTRIES,9377978610,03/26/2021,405.0,PPP,KREMKOV INDUSTRIES LLC,3081 Leatherleaf Trl,Douglasville,...,Unanswered,NaN,NaN,NaN,KREMKOVINDUSTRIES,Kremkov Industries LLC,1,"$9,554,425",30,5/13/2022
63,61,Blessing Box Co LLC,BLESSINGBOXCO,9493207908,06/20/2020,931.0,PPP,BLESSING BOX CO,3400 Cottage Way 2206,Sacramento,...,Unanswered,NaN,NaN,NaN,BLESSINGBOXCO,Blessing Box Co LLC,1,"$1,143,191",31,2/24/2022


In [ ]:
#import confirmed fraud dataset

confirmed_fraud_df = pd.read_csv("PPP_Fraud_Subsample.csv")
confirmed_fraud_df
X = test_df
y= test_df

In [ ]:
#preprocessing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X,y)


ValueError: ignored

In [ ]:
#set split
  #try to follow 90-10 or 80-20 rule depending on data set size
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
y.shape

(63, 56)

In [ ]:
#voting classifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

lr_clf = LogisticRegression(random_state=0)
rf_clf = RandomForestClassifier(max_depth = 5, random_state= 0)
svc_clf= SVC(gamma='auto')
mlp_clf = MLPClassifier(random_state=0, max_iter=1000)
voter= VotingClassifier(estimators=[
    ('lr',lr_clf),('rf',rf_clf),('svc',svc_clf),('mlp',mlp_clf)],voting='hard')
voter.fit(X_train, y_train)

pred=voter.predict(X_test)

TypeError: ignored

In [ ]:
#evaluation
from sklearn.model_selection import metrics
eval_score = cross_val_score(voter,X_train,y_train,cv=4)

In [ ]:
#cross validation

In [ ]:
#confusion matrix